In [ ]:
from preprocess import Preprocess
import pandas as pd

#### Instantiating the Preprocess class...

In [ ]:
preprocessor = Preprocess("Paris")

#### Processing the raw buildings and POIs downloaded from Geofabrik.

In [ ]:
building, poi = preprocessor.get_building_and_poi()

#### Performing Poisson Disk Sampling...

TODO: boundary, buildings e samples vanno proiettate in un CRS metrico. Usare 2154 (Francia)!

In [ ]:
radius = 50
random_point = preprocessor.poisson_disk_sampling(building, poi, radius)